In [3]:
# Script to clean pre-process BIOMASS INVENTORY and make spatial

# First, load packages
import pandas as pd
import os
import numpy as np
import shapely as sp

import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
# only for jupyter nb to show plots inline
#%matplotlib inline 

In [10]:
#check wd
#print(os.getcwd())

#change wd
os.chdir("/Users/anayahall/projects/grapevine")

#read in biomass inventory
# GROSS inventory
gbm = pd.read_csv("data/raw/biomass.inventory.csv")

# TECHNICAL inventory
tbm = pd.read_csv("data/raw/biomass.inventory.technical.csv")

In [50]:
gbm.head()
tbm.head()

gbm.columns

len(gbm.COUNTY.unique())

59

In [13]:
gbm['biomass.category'].value_counts()
# same as technical

row residue                               5900
row culls                                 4425
orchard vineyard residue                  3717
orchard vineyard culls                    3186
field residue                             2655
organic fraction municipal solid waste    2378
low moisture solids                       1947
high moisture solids                      1416
forestry                                   696
manure                                     177
Name: biomass.category, dtype: int64

In [26]:
gbm['biomass.feedstock'].value_counts().head()
# same as technical
# tbm['biomass.feedstock'].value_counts().head()

ONION               413
POTATO              413
SQUASH              413
VEG OTHER           413
SPINACH & GREENS    413
Name: biomass.feedstock, dtype: int64

In [27]:
gbm[gbm['disposal.yields'] == gbm['disposal.yields'].max()]

,COUNTY,biomass.feedstock,disposal.yields,year,biomass.category
26027,Tulare,MANURE,2528938.639,2050,manure


In [53]:
#look at just manure (if feedstock, needs to be capitalized), if category, lower case -- should be equivalent!
gbm[(gbm['biomass.feedstock'] == "MANURE") & (gbm['year'] == 2014)]


,COUNTY,biomass.feedstock,disposal.yields,year,biomass.category
25856,Alameda,MANURE,3.061185e+04,2014,manure
25857,Alpine,MANURE,5.960211e+02,2014,manure
25858,Amador,MANURE,2.828766e+04,2014,manure
25859,Butte,MANURE,3.163301e+04,2014,manure
25860,Calaveras,MANURE,3.061648e+04,2014,manure
25861,Colusa,MANURE,2.958099e+04,2014,manure
25862,Contra Costa,MANURE,4.337103e+04,2014,manure
25863,Del Norte,MANURE,1.850684e+04,2014,manure
25864,El Dorado,MANURE,2.122161e+04,2014,manure
25865,Fresno,MANURE,8.290624e+05,2014,manure


In [56]:
#start grouping by: biomass category

gbm.groupby(['biomass.category'])['disposal.yields'].sum()

biomass.category
field residue                             5.670601e+06
forestry                                  8.363639e+07
high moisture solids                      2.878861e+06
low moisture solids                       1.527702e+07
manure                                    3.440785e+07
orchard vineyard culls                    5.222257e+05
orchard vineyard residue                  1.096371e+07
organic fraction municipal solid waste    6.297142e+07
row culls                                 1.573085e+06
row residue                               1.314184e+06
Name: disposal.yields, dtype: float64

In [62]:
gbm[gbm['biomass.category'] == "manure"].groupby(['COUNTY'])['disposal.yields'].sum()

COUNTY
Alameda            7.838647e+04
Alpine             1.483437e+03
Amador             7.137076e+04
Butte              8.374298e+04
Calaveras          8.102955e+04
Colusa             7.531544e+04
Contra Costa       1.158924e+05
Del Norte          5.757914e+04
El Dorado          5.865816e+04
Fresno             2.731051e+06
Glenn              3.393409e+05
Humboldt           3.276725e+05
Imperial           1.239133e+06
Inyo               7.455241e+04
Kern               2.429723e+06
Kings              2.637177e+06
Lake               2.580007e+04
Lassen             1.933719e+05
Los Angeles        6.690043e+04
Madera             1.191919e+06
Marin              2.058518e+05
Mariposa           7.937785e+04
Mendocino          1.036789e+05
Merced             3.955204e+06
Modoc              2.444919e+05
Mono               2.455981e+04
Monterey           2.395309e+05
Napa               3.480139e+04
Nevada             3.558711e+04
Orange             1.495000e+04
Other              1.050073e+06
P